In [0]:
# Célula Completa: Setup + GBT Challenger
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import GBTClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator

# 1. Ler a tabela Silver novamente
df = spark.read.table("portfolio_credit.german_credit_silver")

# 2. Recriar o Vetor de Features (exatamente como fizemos no Random Forest)
feature_cols = [
    "duration", "credit_amount", "monthly_installment", "age",  
    "checking_status_ix", "credit_history_ix", "purpose_ix", "housing_ix" 
]

assembler = VectorAssembler(inputCols=feature_cols, outputCol="features")
df_model = assembler.transform(df)

# 3. Refazer o Split (70/30)
train_data, test_data = df_model.randomSplit([0.7, 0.3], seed=42)

# 4. Agora sim: Treinar o GBT (Gradient Boosted Trees)
# O GBT constrói árvores uma atrás da outra para corrigir os erros da anterior
gbt = GBTClassifier(labelCol="label", featuresCol="features", maxIter=20, maxDepth=5, seed=42)

print("Treinando o Desafiante (GBT)...")
gbt_model = gbt.fit(train_data)

# 5. Avaliar e Comparar
predictions_gbt = gbt_model.transform(test_data)
evaluator_gbt = BinaryClassificationEvaluator(labelCol="label", rawPredictionCol="rawPrediction", metricName="areaUnderROC")
auc_gbt = evaluator_gbt.evaluate(predictions_gbt)

print(f"\n--- RESULTADO DA BATALHA ---")
print(f"Random Forest (Seu atual): 0.7459")
print(f"GBT (Desafiante):          {auc_gbt:.4f}")

if auc_gbt > 0.7459:
    print("CONCLUSÃO: O GBT venceu! Vale a pena considerar a troca.")
else:
    print("CONCLUSÃO: O Random Forest segurou o cinturão! Ele é mais estável para essa base.")